In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%time
# 大体10分くらい

!pip install -q kaggle
!mkdir -p .kaggle
!cp "./drive/My Drive/Study/config/kaggle.json" .kaggle/
!chmod 600 .kaggle/kaggle.json
!mv .kaggle /root

!kaggle datasets download "theoviel/rcfx-spectrograms-32-khz"
!unzip rcfx-spectrograms-32-khz.zip > /dev/null
!rm -rf rcfx-spectrograms-32-khz.zip 

!pip install -U iterative-stratification albumentations wandb  > /dev/null
!wandb login e0792bb688a0d18e359df7438c45da90f8794091

!pip install timm
!pip install imbalanced-learn

In [ ]:
import gc
import os
import tqdm
import random
import pickle

from matplotlib import pyplot as plt
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from datetime import datetime

import pandas as pd
import numpy as np
from numpy.random import beta

import torch
from torchvision.models import resnet18, densenet121
import torch.nn as nn
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from torchvision import transforms
from torch.nn import functional as F

import albumentations as A

from sklearn.metrics import classification_report, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import wandb
import timm

from sklearn.linear_model import LogisticRegression

device = torch.device("cuda")

In [ ]:
# timm.list_models("resnet*")

In [ ]:
N_EXP = "0171"
#MODEL_NAME = "resnet18"
MODEL_NAME = "efficientnet_b0"
#MODEL_NAME = "densenet121"
#MODEL_NAME = "resnest50d"
DETAIL = "pseudo_loss"
STAGE = "4th"  # 1st, 2nd, 3rd, 4th

#BATCH_SZE = 64
#BATCH_SZE = 32
BATCH_SZE = 8
#N_ACCUMULATE = 1
N_ACCUMULATE = 4

SEED = 416
N_FOLD = 5
WORKS = 0
N_LABEL = 24

EXP_NAME = f"exp{N_EXP}_{MODEL_NAME}_{DETAIL}_{STAGE}"
OUTPUT = f"./drive/MyDrive/Study/RFCX/output/{EXP_NAME}"
DATA_ROOT = "./drive/MyDrive/Study/RFCX/input"

print(EXP_NAME)
!mkdir -p {OUTPUT}

データ読み込み

In [ ]:
# Competition Data
sample_submission = pd.read_csv(f"{DATA_ROOT}/sample_submission.csv")
train_fp = pd.read_csv(f"{DATA_ROOT}/train_fp.csv")
train_tp = pd.read_csv(f"{DATA_ROOT}/train_tp.csv")

# Pseudo Label
if STAGE == "3rd":
    PSEUDO = "exp0127_resnet18_second_st_only_label_loss_thr0.5"
    with open(f"./drive/MyDrive/Study/RFCX/pseudo_labels/{PSEUDO}.pkl", "rb") as f:
        pseudo_label_dict_5fold = pickle.load(f)
elif STAGE == "4th":
    PSEUDO = "exp0153_resnet18_focal_mixup_pseudo0.5_thr0.5"
    with open(f"./drive/MyDrive/Study/RFCX/pseudo_labels/{PSEUDO}.pkl", "rb") as f:
        pseudo_label_dict_5fold = pickle.load(f)

# Pre-Train Model
if MODEL_NAME == "resnet18":
    FIRST_ST = "exp0109_only_clip_loss"
elif MODEL_NAME == "densenet121":
    FIRST_ST = "exp0157_densenet121_for_ensemble_1st"
elif MODEL_NAME == "resnest50d":
    FIRST_ST = "exp0117_resnest_now_best"
elif MODEL_NAME == "efficientnet_b0":
    FIRST_ST = "exp0158_efficientnet_b0_for_ensemble_1st"

# Parameter
if STAGE == "1st":
    LEARNING_RATE = 1e-3
    NUM_EPOCHS = 50
    T_MAX = 10
else:
    LEARNING_RATE = 3e-4
    NUM_EPOCHS = 5
    T_MAX = 5

utils

In [ ]:
# https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/198418

def LWLRAP(preds, labels):
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

ラベルの抽出

In [ ]:
tp_dict = {}
for recording_id, df in train_tp.groupby("recording_id"):
    tp_dict[recording_id+"_posi"] = df.values[:, [1,3,4,5,6]]

fp_dict = {}
for recording_id, df in train_fp.groupby("recording_id"):
    fp_dict[recording_id+"_nega"] = df.values[:, [1,3,4,5,6]]

tp_fnames, tp_labels = [], []
for recording_id, df in train_tp.groupby("recording_id"):
    v = sum([np.eye(N_LABEL)[i] for i in df["species_id"].tolist()])
    v = (v  == 1).astype(int).tolist()
    tp_fnames.append(recording_id+"_posi")
    tp_labels.append(v)

fp_fnames, fp_labels = [], []
for recording_id, df in train_fp.groupby("recording_id"):
    v = sum([np.eye(N_LABEL)[i] for i in df["species_id"].tolist()])
    v = (v  == 1).astype(int).tolist()
    fp_fnames.append(recording_id+"_nega")
    fp_labels.append(v)

test_fnames = sample_submission["recording_id"].values

DataLoader

In [ ]:
WINDOW = 512
MAX_LEN = 3751
SHIFT_W = [0, 32, 64, 128, 256]

def extract_cut_pos(pos):

    _shift_w = random.choice(SHIFT_W)
    _shift_dist = random.choice([1, -1])
    _pos = pos + (_shift_w * _shift_dist)

    h_pos = _pos - 256
    t_pos = _pos + 256
    if h_pos <= 0:
        h_pos, t_pos = 0, WINDOW
    if t_pos >= MAX_LEN:
        h_pos, t_pos = MAX_LEN-WINDOW, MAX_LEN
    return h_pos, t_pos

slide_img_pos = [[0, WINDOW]]
for idx in range(1, 8):
    h, t = slide_img_pos[idx-1][0], slide_img_pos[idx-1][1]
    h = t - 49
    t = h + WINDOW
    slide_img_pos.append([h, t])

print(slide_img_pos)

In [ ]:
class TimeMask:
    def __init__(self, T=40, num_masks=1, replace_with_zero=True):
        self.T = T
        self.num_masks = num_masks
        self.replace_with_zero = replace_with_zero

    def __call__(self, spec):
        cloned = spec.clone()
        len_spectro = cloned.shape[2]
    
        for i in range(0, self.num_masks):
            t = random.randrange(0, self.T)
            t_zero = random.randrange(0, len_spectro - t)

            # avoids randrange error if values are equal and range is empty
            if (t_zero == t_zero + t): return cloned

            mask_end = random.randrange(t_zero, t_zero + t)
            if (self.replace_with_zero): cloned[:,:,t_zero:mask_end] = 0
            else: cloned[:,:,t_zero:mask_end] = cloned.mean()
        return cloned

class FreqMask:
    def __init__(self, F=30, num_masks=1, replace_with_zero=True):
        self.F = F
        self.num_masks = num_masks
        self.replace_with_zero = replace_with_zero

    def __call__(self, spec):
        cloned = spec.clone()
        num_mel_channels = cloned.shape[1]
    
        for i in range(0, self.num_masks):        
            f = random.randrange(0, self.F)
            f_zero = random.randrange(0, num_mel_channels - f)

            # avoids randrange error if values are equal and range is empty
            if (f_zero == f_zero + f): return cloned

            mask_end = random.randrange(f_zero, f_zero + f) 
            if (self.replace_with_zero): cloned[:, f_zero:mask_end] = 0
            else: cloned[:, f_zero:mask_end] = cloned.mean()
    
        return cloned

def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def extract_seq_label(label, value):
    seq_label = np.zeros((N_LABEL, 3751))
    middle = np.ones(N_LABEL) * -1
    for species_id, t_min, f_min, t_max, f_max in label:
        h, t = int(3751*(t_min/60)), int(3751*(t_max/60))
        m = (t + h)//2
        middle[species_id] = m
        seq_label[species_id, h:t] = value
    return seq_label, middle.astype(int)

class SpectrogramFromNpz(torch.utils.data.Dataset):
    def __init__(self, fname, mode):
        self.fname = fname
        self.mode = mode
        self.to_tensor = transforms.ToTensor()
        self.norm = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.augument_funcs_a = A.Compose([
            # A.RandomBrightnessContrast(p=0.5)
        ])
        self.augument_funcs_b = transforms.RandomApply([
            #transforms.Lambda(lambda img: transforms.functional.adjust_gamma(img, gamma=2, gain=1)),
            TimeMask(),
            FreqMask(),
        ], p=0.5)

    def __len__(self):
        return len(self.fname)

    def __getitem__(self, idx):
        fname = self.fname[idx]

        # load label
        if fname[-4:] == "posi":
            label, middle = extract_seq_label(tp_dict[fname], 1)
        elif fname[-4:] == "nega": 
            label, middle = extract_seq_label(fp_dict[fname], -1)
        else:
            label, middle = np.zeros((N_LABEL, 3751)), np.zeros(N_LABEL)

        # load image
        if self.mode in ["train", "valid"]:
            if "_posi" in fname or "_nega" in fname:
                img_dir = "train"
            else:
                img_dir = "test"
        elif self.mode == "test":
            img_dir = "test"
        
        _fname = fname.replace("_posi", "").replace("_nega", "")
        path = f"./{img_dir}/{_fname}.npy"
        mel = np.load(path)
        image = mono_to_color(mel)

        # augument
        if self.mode == "train":
            image = self.augument_funcs_a(image=image)["image"]
            image = self.to_tensor(image)
            image = self.augument_funcs_b(image)
        else:
            image = self.to_tensor(image)
        image = self.norm(image)

        # pseudo
        try:
            pseudo_label = pseudo_label_dict_5fold[fname]
        except KeyError:
            pseudo_label = np.zeros((8, N_LABEL))
        except NameError:
            pseudo_label = np.zeros((8, N_LABEL))

        return image, label, middle, pseudo_label

In [ ]:
#_datasets = SpectrogramFromNpz(tp_fnames[:2], "train")
#_datasets = SpectrogramFromNpz(fp_fnames[:2], "train")
#_datasets = SpectrogramFromNpz(test_fnames.tolist(), "train")
#for d in _datasets:
   #break
   #pass

In [ ]:
#plt.imshow(d[0].transpose(0, 2).transpose(1, 0).numpy()); plt.show()
#plt.plot(d[1].sum(0)); plt.show()
#org_mage = mono_to_color(np.load("train/003bec244.npy"))
#plt.imshow(org_mage[:,2770:3282,:]); plt.show()

In [ ]:
def rfcx_1st_criterion(outputs, targets, pseudo, b, perm):
    clipwise_preds_att_ti = outputs["clipwise_preds_att_ti"]
    posi_label = ((targets == 1).sum(2) > 0).float().to(device)
    posi_label = mixup(posi_label, b, perm)
    loss = nn.BCEWithLogitsLoss(reduction="mean")(clipwise_preds_att_ti, posi_label)
    return loss

def rfcx_2nd_criterion(outputs, targets, pseudo=None, b=None, perm=None):
    clipwise_preds_att_ti = outputs["clipwise_preds_att_ti"]
    posi_label = ((targets == 1).sum(2) > 0).float().to(device)
    nega_label = ((targets == -1).sum(2) > 0).float().to(device)
    posi_y = torch.ones(clipwise_preds_att_ti.shape).to(device)
    nega_y = torch.zeros(clipwise_preds_att_ti.shape).to(device)
    posi_loss = nn.BCEWithLogitsLoss(reduction="none")(clipwise_preds_att_ti, posi_y)
    nega_loss = nn.BCEWithLogitsLoss(reduction="none")(clipwise_preds_att_ti, nega_y)
    posi_loss = (posi_loss * posi_label).sum()
    nega_loss = (nega_loss * nega_label).sum()
    loss = posi_loss + nega_loss
    return loss

def rfcx_3rd_criterion(outputs, targets, pseudo, b, perm):
    clipwise_preds_att_ti = outputs["clipwise_preds_att_ti"]

    posi_pseudo = (pseudo > 0).int().to(device)
    posi_label = ((targets == 1).sum(2) + posi_pseudo > 0).float().to(device)
    nega_label = ((targets == -1).sum(2) > 0).float().to(device)

    posi_y = torch.ones(clipwise_preds_att_ti.shape).to(device)
    nega_y = torch.zeros(clipwise_preds_att_ti.shape).to(device)

    # mixup treat
    posi_label = mixup(posi_label, b, perm)
 
    posi_loss = nn.BCEWithLogitsLoss(reduction="none")(clipwise_preds_att_ti, posi_y)
    nega_loss = nn.BCEWithLogitsLoss(reduction="none")(clipwise_preds_att_ti, nega_y)

    gamma = 2
    probas = clipwise_preds_att_ti.sigmoid()
    focal_pw = (1. - probas)**gamma

    posi_loss = (posi_loss * posi_label * focal_pw).sum()
    nega_loss = (nega_loss * nega_label).sum()

    pos_w = 0 if posi_label.sum() == 0 else 1/posi_label.sum()
    loss = posi_loss*pos_w + nega_loss

    return loss

def mixup(input, gamma, perm):
    perm_input = input[perm]
    return input.mul_(gamma).add_(1 - gamma, perm_input)

def cut_and_sampling_img(X, y, m):
    X_lst, y_lst = [], []
    for _i, _m in enumerate(m):
        idx = np.where(_m >= 0)
        pos = _m[idx[0]]
        pos = random.choice(pos)
        h, t = extract_cut_pos(pos)
        _X = X[_i, :, :, h:t]
        _y = y[_i, :, h:t]
        X_lst.append(_X)
        y_lst.append(_y)
    _X = torch.stack(X_lst)
    _y = torch.stack(y_lst)
    return _X, _y

def split_and_padding(X, y):
    x_lst = []
    y_lst = []
    for h, t in slide_img_pos:
        _X = X[:, :, :, h:t]
        _y = y[:, :, h:t]
        if _X.shape[3] != WINDOW:
            x_pad = torch.zeros(list(_X.shape[:-1]) + [WINDOW - _X.shape[3]])
            _X = torch.cat([_X, x_pad], axis=3)
            y_pad = torch.zeros(list(_y.shape[:-1]) + [WINDOW - _y.shape[2]])
            _y = torch.cat([_y, y_pad], axis=2)
        x_lst.append(_X)
        y_lst.append(_y)
    
    X = torch.cat(x_lst, axis=0)
    y = torch.cat(y_lst, axis=0)
    return X, y

def extract_pseudo_label(p):
    p_lst = []
    for i in range(8):
        p_lst.append(p[:, i, :])
    pseudo_y = torch.cat(p_lst, axis=0)
    return pseudo_y

def train_loop(train_data_loader, model, optimizer, scheduler, stage):
    model.train()
    losses, lrs = [], []

    optimizer.zero_grad()
    for n_iter, (X, y, m, p) in tqdm.tqdm_notebook(enumerate(train_data_loader), total=len(train_data_loader)):
        if stage == "1st":
            _X, _y = cut_and_sampling_img(X, y, m)
        elif stage in ["2nd", "3rd", "4th"]:
            _X, _y = split_and_padding(X, y)
        _X, _y = _X.to(device), _y.to(device)

        if stage in ["3rd", "4th"]:
            pseudo_y = extract_pseudo_label(p)
        elif stage in ["1st", "2nd"]:
            pseudo_y = None

        # mixup
        b = beta(0.1, 0.1)
        perm = torch.randperm(_X.size(0))
        if stage in ["1st", "3rd"]:
            _X = mixup(_X, b, perm)
        
        if stage == "4th":
            outputs = model(_X, perm, b)
        else:
            outputs = model(_X)

        loss = rfcx_criterion(outputs, _y, pseudo_y, b, perm)

        loss.backward()
        if n_iter % N_ACCUMULATE == 0:
            optimizer.step()
            optimizer.zero_grad()

        if scheduler is not None:
            scheduler.step()

        losses.append(loss.item())
        lrs.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())
    t_loss = np.array(losses).mean()
    lr =  np.array(lrs).mean()
    return t_loss, lr
    
def valid_loop(valid_data_loader, model, stage):
    model.eval()
    v_scores, v_losses = [], []
    clip_preds, clip_targets = [], []
    for X, y, m, p in valid_data_loader:
        X = X.to(device)
        if stage in ["1st", "2nd"]:
            clip_y = ((y == 1).sum(2) > 0).float().to(device)
        elif stage in ["3rd", "4th"]:
            clip_y = ((y == 1).sum(2) + (p == 1).sum(1) > 0).float().to(device)
        
        preds = []
        for h, t in slide_img_pos:
            with torch.no_grad():
                outputs = model(X[:,:,:,h:t])
            pred, _ = outputs["segmentwise_output_ti"].sigmoid().max(2)
            preds.append(pred)
        max_pred, _  = torch.max(torch.stack(preds), dim=0)
            
        score = LWLRAP(max_pred.cpu(), clip_y.cpu())
        loss = nn.BCEWithLogitsLoss()(max_pred, clip_y).cpu().numpy()
        v_scores.append(score)
        v_losses.append(loss.item())

        clip_preds.append(max_pred.cpu())
        clip_targets.append(clip_y.cpu())

    # calc precision
    pred_y = torch.cat(clip_preds)
    true_y = torch.cat(clip_targets)
    lst = []
    for _true_y, _pred_y in zip(true_y.T, pred_y.T):
        res = classification_report(_true_y.int(), (_pred_y > 0.5).int(), output_dict=True)
        res = res["1"]
        lst.append(res)
    res_df = pd.DataFrame(lst)
    precision_avg = res_df["precision"].mean()

    valid_score, valid_loss = np.array(v_scores).mean(), np.array(v_losses).mean()
    return valid_score, valid_loss, precision_avg

In [ ]:
# ヘッダーのインデックスと次元数
MODEL_HEADER_INFO = {
    "resnet18": (-2, 512),
    "densenet121": (-2, 1024),
    "efficientnet_b0": (-5, 320),
    "resnest50d": (-2, 2048),
    "mobilenetv2_100": (-2, 1280),
}

def interpolate(x: torch.Tensor, ratio: int):
    x = x.transpose(1, 2)
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    upsampled = upsampled.transpose(1, 2)
    return upsampled

class RFCXNet(nn.Module):
    def __init__(self, model_name):
        super(RFCXNet, self).__init__()
        self.model_name = model_name
        self.n_label = N_LABEL

        base_model = timm.create_model(model_name, pretrained=True)
        h_idx, n_dense = MODEL_HEADER_INFO[model_name]        

        # 過去学習に使ったモデルをロードするためヘッダーの名前を変える
        if self.model_name in ["resnet18", "efficientnet_b0"]:
            self.resnet_head = nn.Sequential(*list(base_model.children())[:h_idx])
        elif self.model_name == "resnest50d":
            self.resnest50d_head = nn.Sequential(*list(base_model.children())[:h_idx])
        else:
            self.model_head = nn.Sequential(*list(base_model.children())[:h_idx])
                
        self.fc_a = nn.Conv1d(n_dense, self.n_label, 1, bias=False)
        self.fc_b = nn.Conv1d(n_dense, self.n_label, 1, bias=False)

    def forward(self, x, perm=None, gamma=None):  # input x: (batch, channel, Hz, time)
        frames_num = x.shape[3]
        x = x.transpose(3, 2)  # (batch, channel, time, Hz)

        # (batch, unit, time, Hz)
        if self.model_name in ["resnet18", "efficientnet_b0"]:
            h = self.resnet_head(x)  
        elif self.model_name == "resnest50d":
            h = self.resnest50d_head(x)
        else:
            h = self.model_head(x)
        
        if perm is not None:
            h = gamma * h + (1 - gamma) * h[perm]
            
        h = F.relu(h)
        ti_pool = torch.mean(h, dim=3)  # (batch, unit, time)

        xa = self.fc_a(ti_pool)  # (batch, n_class, time)
        xb = self.fc_b(ti_pool)  # (batch, n_class, time)
        xb = torch.softmax(xb, dim=2)

        # time pool
        # pseudo_label_ti = ((xa.sigmoid() >= 0.5) > 0).float()
        clipwise_preds_att_ti = torch.sum(xa * xb, dim=2)
        # attention_preds_ti = xb
        segmentwise_output_ti = interpolate(xa, 32)

        return {
            # "pseudo_label_ti": pseudo_label_ti,
            "clipwise_preds_att_ti": clipwise_preds_att_ti,
            # "attention_preds_ti": attention_preds_ti,
            "segmentwise_output_ti": segmentwise_output_ti,
        }

In [ ]:
#train_datasets = SpectrogramFromNpz(tp_fnames, "train")
#train_data_loader = torch.utils.data.DataLoader(train_datasets, batch_size=2, shuffle=True, num_workers=0)
#for n_iter, (X, y, m, p) in tqdm.tqdm_notebook(enumerate(train_data_loader)):
#    _X, _y = split_and_padding(X, y)
#    _X, _y = cut_and_sampling_img(X, y, m)
#    preds = []
#    for h, t in eval_img_pos:
#        break
    #break

In [ ]:
#model = RFCXNet("resnet18")
#model.to(device)
#model.eval()
#with torch.no_grad():
    #outputs = model(X[:,:,h:t].unsqueeze(0))
    #outputs = model(_X.to(device))
#outputs["segmentwise_output_ti"].shape

# Train

In [ ]:
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
test_datasets = SpectrogramFromNpz(test_fnames, "test")
for fold, (train_index, valid_index) in enumerate(mskf.split(tp_fnames, tp_labels)):
    if fold in [0]:
        continue
    print(datetime.now(), f"\t: ### FOLD-{fold} ###")
    set_seed(SEED+fold)

    wandb.init(project="rfcx", name=f"{EXP_NAME}_f{fold}")

    config = wandb.config
    config.exp_name = EXP_NAME
    config.fold = fold
    config.seed = SEED
    config.learning_rate = LEARNING_RATE
    config.batch_size = BATCH_SZE
    config.num_epochs = NUM_EPOCHS
    config.t_max = T_MAX
    config.n_accumulate = N_ACCUMULATE

    train_fname = np.array(tp_fnames)[train_index]
    valid_fname = np.array(tp_fnames)[valid_index]

    if STAGE == "1st":
        train_datasets = SpectrogramFromNpz(train_fname.tolist()+fp_fnames[:30], "train")
        rfcx_criterion = rfcx_1st_criterion
    elif STAGE == "2nd":
        train_datasets = SpectrogramFromNpz(train_fname.tolist()+fp_fnames, "train")
        rfcx_criterion = rfcx_2nd_criterion
    elif STAGE in ["3rd", "4th"]:
        train_datasets = SpectrogramFromNpz(train_fname.tolist()+fp_fnames, "train")
        rfcx_criterion = rfcx_3rd_criterion
    
    train_data_loader = torch.utils.data.DataLoader(train_datasets, batch_size=config.batch_size, shuffle=True, num_workers=WORKS)
    valid_datasets = SpectrogramFromNpz(valid_fname, "valid")
    valid_data_loader = torch.utils.data.DataLoader(valid_datasets, batch_size=config.batch_size, shuffle=False, num_workers=WORKS)

    model = RFCXNet(MODEL_NAME)
    if STAGE in ["2nd", "3rd", "4th"]:
        model.load_state_dict(torch.load(f"./drive/MyDrive/Study/RFCX/output/{FIRST_ST}/rfcxnet_f{config.fold}_best_score_model.bin"))
    model.to(device)

    optimizer = Adam(model.parameters(), lr=config.learning_rate)
    scheduler = CosineAnnealingLR(optimizer, T_max=len(train_data_loader)*config.t_max, eta_min=0.0)

    wandb.watch(model)

    print(datetime.now(), "\t: start train")
    best_score, best_loss, best_precision = 0, 9999, 0
    for epoch in range(config.num_epochs):
        if STAGE == "1st":
            del train_datasets, train_data_loader
            _fp_fnames = random.sample(fp_fnames, 30)
            train_datasets = SpectrogramFromNpz(train_fname.tolist()+_fp_fnames, "train")
            train_data_loader = torch.utils.data.DataLoader(train_datasets, batch_size=config.batch_size, shuffle=True, num_workers=WORKS)
            gc.collect()

        t_loss, lr = train_loop(train_data_loader, model, optimizer, scheduler, STAGE)
        v_score, v_loss, precision_avg = valid_loop(valid_data_loader, model, STAGE)

        if best_score < v_score:
            print(f"epoch {epoch}: best score update !!!")
            torch.save(model.state_dict(), f"{OUTPUT}/rfcxnet_f{config.fold}_best_score_model.bin")
            best_score = v_score

        if best_precision < precision_avg:
            print(f"epoch {epoch}: best precision update !!!")
            torch.save(model.state_dict(), f"{OUTPUT}/rfcxnet_f{config.fold}_best_precision_model.bin")
            best_precision = precision_avg

        if best_loss > v_loss:
            print(f"epoch {epoch}: best loss update !!!")
            torch.save(model.state_dict(), f"{OUTPUT}/rfcxnet_f{config.fold}_best_loss_model.bin")
            best_loss = v_loss
    
        wandb.log({"train loss": t_loss, "lr": lr, "valid loss": v_loss, "valid score": v_score, "best score": best_score, "best loss": best_loss,
                   "precision": precision_avg, "best_precision": best_precision})

    print(datetime.now(), "\t: finish train")
    print("Best Valid Score:", best_score)
    wandb.finish()

    # predict test data
    model.load_state_dict(torch.load(f"{OUTPUT}/rfcxnet_f{config.fold}_best_score_model.bin"))
    model.eval()

    lst = []
    for idx, (X, _, _, _) in tqdm.tqdm_notebook(enumerate(test_datasets), total=1992):
        preds = []
        for h, t in slide_img_pos:
            with torch.no_grad():
                outputs = model(X[:,:,h:t].unsqueeze(0).to(device))
            pred, _ = outputs["segmentwise_output_ti"].sigmoid().max(2)
            preds.append(pred)
        max_pred, _  = torch.max(torch.stack(preds), dim=0)
        pred = max_pred.cpu().numpy()[0].tolist()

        row = [test_fnames[idx]] + pred
        lst.append(row)

    fold_sub = pd.DataFrame(lst, columns=["recording_id"]+[f"s{i}" for i in range(N_LABEL)])
    fold_sub.to_csv(f"{OUTPUT}/rfcxnet_f{config.fold}_predict.csv", index=None)

# Test

In [ ]:
all_v_lst = []
for fold in range(5):
    df = pd.read_csv(f"{OUTPUT}/rfcxnet_f{fold}_predict.csv")
    ids, v_lst = [], []
    for row in df.values:
        recording_id = row[0]
        ids.append(recording_id)
        v = torch.Tensor(row[1:].astype(float))
        v_lst.append(v)
    all_v_lst.append(torch.stack(v_lst, axis=0))

all_preds = torch.stack(all_v_lst, axis=2).mean(2)
sub = pd.DataFrame(all_preds.tolist(), columns=df.columns[1:])
sub = pd.concat([df[["recording_id"]], sub], axis=1)
sub.to_csv(f"./submission_{EXP_NAME}_avg.csv", index=None)

!cp "./submission_{EXP_NAME}_avg.csv" "{OUTPUT}"

# Pseudo

In [ ]:
#EXP = "exp0127_resnet18_second_st_only_label_loss"
EXP = "exp0153_resnet18_focal_mixup_pseudo0.5"
# EXP = EXP_NAME
PSEUDO_THR_P = 0.5
PSEUDO_THR_N = 0.01
BEST_MODEL = f"./drive/MyDrive/Study/RFCX/output/{EXP}"

In [ ]:
model = RFCXNet(MODEL_NAME)
model.to(device)
all_fnames = tp_fnames+fp_fnames
train_datasets = SpectrogramFromNpz(all_fnames, "valid")
#test_datasets = SpectrogramFromNpz(test_fnames, "test")

In [ ]:
pseudo_label_dict_lst = []
for fold in range(5):
    print(f"### {fold} ###")

    model.load_state_dict(torch.load(f"{BEST_MODEL}/rfcxnet_f{fold}_best_score_model.bin"))
    model.eval()

    pseudo_label_dict = {}
    for fname, (X, y, m, _) in tqdm.tqdm_notebook(zip(all_fnames, train_datasets), total=len(train_datasets)):
    #for fname, (X, y, m, _) in tqdm.tqdm_notebook(zip(test_fnames, test_datasets), total=len(test_datasets)):
        labels = []
        posi_labels, nega_labels = [], []
        for h, t in slide_img_pos:
            with torch.no_grad():
                outputs = model(X[:,:,h:t].unsqueeze(0).to(device))
            pred = outputs["clipwise_preds_att_ti"].sigmoid().cpu().numpy()[0]
            
            posi = (pred > PSEUDO_THR_P).astype(int)
            nega = (pred < PSEUDO_THR_N).astype(int) * -1
            posi_labels.append(posi)
            nega_labels.append(nega)

        posi_labels = np.stack(posi_labels).astype(int)
        nega_labels = np.stack(nega_labels).astype(int)
        if posi_labels.sum() == 0 and nega_labels.sum() == 0:
            continue
        labels = np.stack([posi_labels, nega_labels])
        pseudo_label_dict[fname] = labels
    pseudo_label_dict_lst.append(pseudo_label_dict)

In [ ]:
pseudo_label_dict_5fold = {}
for fname in all_fnames:
#for fname in test_fnames:
    labels = []
    for pseudo_label_dict in pseudo_label_dict_lst:
        try:
            label = pseudo_label_dict[fname]
        except KeyError:
            label = np.zeros((2, 8, N_LABEL))
        labels.append(label)

    posi_label, nega_label = np.stack(labels).sum(0)
    posi_label = (posi_label > 2).astype(int)
    nega_label = (nega_label < -2).astype(int) * -1
    new_label = posi_label + nega_label

    if (new_label != 0).sum() == 0:
        continue
    pseudo_label_dict_5fold[fname] = new_label

In [ ]:
lst = [(v==1).sum(0) > 0 for v in pseudo_label_dict_5fold.values()]
pd.DataFrame(lst).sum(0).plot.bar()

In [ ]:
with open(f"./drive/MyDrive/Study/RFCX/pseudo_labels/{EXP}_thr{PSEUDO_THR_P}.pkl", "wb") as f:
#with open(f"./drive/MyDrive/Study/RFCX/pseudo_labels/{EXP}_test.pkl", "wb") as f:
    pickle.dump(pseudo_label_dict_5fold, f)
    
!ls ./drive/MyDrive/Study/RFCX/pseudo_labels/

# emsemble

In [ ]:
exp_lst = [
    "exp0167_resnet18_mixup_lastlayer_3rd",
    "exp0166_densenet121_mixup_lastlayer_4th",
    "exp0167_efficientnet_b0_mixup_lastlayer_3rd",
]

重み付け

In [ ]:
values = []
for exp in exp_lst:
    csv_path = f"./drive/MyDrive/Study/RFCX/output/{exp}/submission_{exp}_avg.csv"
    df = pd.read_csv(csv_path)
    values.append(df.values[:, 1:])

values = np.stack(values)
df[[f"s{i}" for i in range(24)]] = values.mean(0)
df.to_csv("ensemble_resnet18_densenet121_efficientnet_b0_.csv", index=None)

## Stacking

In [ ]:
def make_validation_prediction(valid_data_loader, model):
    pred_y_lst, true_y_lst = [], []
    for X, y, m, p in tqdm.tqdm_notebook(valid_data_loader):
        X = X.to(device)
        
        # ラベルを予測
        pred_y, true_y = [], []
        for h, t in slide_img_pos:
            with torch.no_grad():
                outputs = model(X[:,:,:,h:t])
            pred, _ = outputs["segmentwise_output_ti"].sigmoid().max(2)
            pred_y.append(pred)
            t = ((y[:, :, h:t] > 0).sum(2) > 0).float()
            true_y.append(t)
        pred_y = torch.stack(pred_y).transpose(0, 1)
        true_y = torch.stack(true_y).transpose(0, 1)

        # pseudをつける
        true_y = ((true_y + (p > 0).int()) > 0).int()

        pred_y = pred_y.reshape((-1, 24))
        true_y = true_y.reshape((-1, 24))

        pred_y_lst.append(pred_y)
        true_y_lst.append(true_y)
    stacking_pred_y = torch.cat(pred_y_lst)
    stacking_true_y = torch.cat(true_y_lst)
    return stacking_pred_y, stacking_true_y

In [ ]:
def stacking_lr_model(train_x, train_y):
    models = []
    lst, scores = [], []
    mskf_stacking = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for trn_i, val_i in mskf_stacking.split(train_x, train_y):
        preds, targets = [], []
        _models = []
        for _id in range(24):
            lr = LogisticRegression() 
            trn_x = train_x[trn_i, :] #[:, [0+_id, 24+_id, 48+_id]]
            trn_y = train_y[trn_i, _id]
            val_x = train_x[val_i, :] #[:, [0+_id, 24+_id, 48+_id]]
            val_y = train_y[val_i, _id]
            lr.fit(trn_x, trn_y)
            val_pred = lr.predict_proba(val_x)[:, 1]
            preds.append(val_pred)
            targets.append(val_y)
            res = classification_report(val_y, (val_pred>0.5).astype(int), output_dict=True)["1"]
            lst.append(res)
            _models.append(lr)
        preds = np.vstack(preds).reshape((-1, 24))
        targets = np.vstack(targets).reshape((-1, 24))
        score = LWLRAP(torch.tensor(preds), torch.tensor(targets))
        scores.append(score)
        models.append(_models)
    results = pd.DataFrame(lst).mean(0)
    results["LWLRAP"] = sum(scores)/5
    return models, results

In [ ]:
test_preds, results_lst = [], []
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
for fold, (train_index, valid_index) in enumerate(mskf.split(tp_fnames, tp_labels)):
    print(f"### Fold-{fold} ###")
    valid_fname = np.array(tp_fnames)[valid_index]
    valid_datasets = SpectrogramFromNpz(valid_fname, "valid")
    valid_data_loader = torch.utils.data.DataLoader(valid_datasets, batch_size=32, shuffle=False, num_workers=WORKS)

    stacking_pred_y_lst = []
    values = []
    for exp in exp_lst:
        model_name = exp.split("_")[1]
        if "efficientnet" in model_name:
            model_name += "_" + exp.split("_")[2]
        print(model_name)
        model = RFCXNet(model_name)
        model.to(device)
        model.load_state_dict(torch.load(f"./drive/MyDrive/Study/RFCX/output/{exp}/rfcxnet_f{fold}_best_score_model.bin"))
        model.eval()
        stacking_pred_y, stacking_true_y = make_validation_prediction(valid_data_loader, model)
        stacking_pred_y_lst.append(stacking_pred_y)

        csv_path = f"./drive/MyDrive/Study/RFCX/output/{exp}/submission_{exp}_avg.csv"
        test_df = pd.read_csv(csv_path)
        values.append(test_df.values[:, 1:])

    train_x = torch.hstack(stacking_pred_y_lst).cpu().numpy()
    train_y = stacking_true_y.numpy()
    models, results = stacking_lr_model(train_x, train_y)
    display(results)
    results_lst.append(results)

    for _models in models:
        preds = []
        for _id, m in enumerate(_models):
            _pred = m.predict_proba(np.hstack(values))[:, 1]
            preds.append(_pred)
        test_preds.append(np.stack(preds).T)

test_preds_avg = np.stack(test_preds).mean(0)
sum(results_lst)/5

test_df[[f"s{i}" for i in range(24)]] = test_preds_avg
test_df.to_csv(f"ensemble_stacking.csv", index=None)

In [ ]:
for exp in exp_lst:
    csv_path = f"./drive/MyDrive/Study/RFCX/output/{exp}/submission_{exp}_avg.csv"
    df1 = pd.read_csv(csv_path)
    break
df2 = pd.read_csv("ensemble_resnet18_densenet121_efficientnet_b0_.csv")
df3 = pd.read_csv("ensemble_stacking.csv")

In [ ]:
for recording_id in df1["recording_id"].iloc[1:]:
    v1 = df1.query(f"recording_id=='{recording_id}'").values[0, 1:]
    v2 = df2.query(f"recording_id=='{recording_id}'").values[0, 1:]
    v3 = df3.query(f"recording_id=='{recording_id}'").values[0, 1:]
    break

In [ ]:
plt.plot(v1)
plt.plot(v2)
plt.plot(v3)

In [ ]:
np.argsort(v1)

In [ ]:
np.argsort(v2)

In [ ]:
np.argsort(v3)